<a href="https://colab.research.google.com/github/Marvin2798/lab-langsmith-summarizations/blob/main/lab-langsmith-sumarizations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab | Summarization evaluation using LangSmith
Let's revisit your capstone project 2? Well, sort of. Pick diffierent sets of data and re-run this notebook. Maybe parts of the dataset you used in your last project week. The point is for you to understand all steps involve and the many different ways one can and should evaluate LLM applications using LangSmith.

What did you learn? - Let's discuss that in class

## LangSmith - LangChain evaluation

In [2]:
!pip install python-dotenv

In [3]:
from dotenv import load_dotenv, find_dotenv
import os
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [4]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]="langsmith_max-test"

In [5]:
#Importing Client from Langsmith
from langsmith import Client
client = Client(api_key=LANGCHAIN_API_KEY)

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


### Create Dataset


In [7]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [8]:
from datasets import load_dataset
cnn_dataset = load_dataset(
    "ccdv/cnn_dailymail", version
    ="3.0.0",
    trust_remote_code=True
)

README.md:   0%|          | 0.00/13.9k [00:00<?, ?B/s]

cnn_dailymail.py:   0%|          | 0.00/9.27k [00:00<?, ?B/s]

cnn_stories.tgz:   0%|          | 0.00/159M [00:00<?, ?B/s]

dailymail_stories.tgz:   0%|          | 0.00/376M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [9]:
def add_prefix(example):
    return {
        **example,
        "article": f"Summarize this news:\n{example['article']}"
    }

#cnn_dataset = cnn_dataset.map(add_prefix)

In [10]:
cnn_dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [11]:
cnn_dataset['train'][0]

{'article': 'It\'s official: U.S. President Barack Obama wants lawmakers to weigh in on whether to use military force in Syria. Obama sent a letter to the heads of the House and Senate on Saturday night, hours after announcing that he believes military action against Syrian targets is the right step to take over the alleged use of chemical weapons. The proposed legislation from Obama asks Congress to approve the use of military force "to deter, disrupt, prevent and degrade the potential for future uses of chemical weapons or other weapons of mass destruction." It\'s a step that is set to turn an international crisis into a fierce domestic political battle. There are key questions looming over the debate: What did U.N. weapons inspectors find in Syria? What happens if Congress votes no? And how will the Syrian government react? In a televised address from the White House Rose Garden earlier Saturday, the president said he would take his case to Congress, not because he has to -- but bec

In [12]:
#Get just a few news to test
MAX_NEWS=10
sample_cnn = cnn_dataset["test"].select(range(MAX_NEWS)).map(add_prefix)

sample_cnn

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 10
})

The dataset contains three columns: article, highlights, and id. To use LangSmith, we need to create a dataset in LangSmith format.

LangSmith expects a prompt and a result. To achieve this, we will transform the article into a prompt by adding the prefix: "Summarize this news." As a result, we will use the content of highlights, which represents the summaries created by humans.

In [13]:
print(sample_cnn[0])

{'article': 'Summarize this news:\n(CNN)James Best, best known for his portrayal of bumbling sheriff Rosco P. Coltrane on TV\'s "The Dukes of Hazzard," died Monday after a brief illness. He was 88. Best died in hospice in Hickory, North Carolina, of complications from pneumonia, said Steve Latshaw, a longtime friend and Hollywood colleague. Although he\'d been a busy actor for decades in theater and in Hollywood, Best didn\'t become famous until 1979, when "The Dukes of Hazzard\'s" cornpone charms began beaming into millions of American homes almost every Friday night. For seven seasons, Best\'s Rosco P. Coltrane chased the moonshine-running Duke boys back and forth across the back roads of fictitious Hazzard County, Georgia, although his "hot pursuit" usually ended with him crashing his patrol car. Although Rosco was slow-witted and corrupt, Best gave him a childlike enthusiasm that got laughs and made him endearing. His character became known for his distinctive "kew-kew-kew" chuckle

Now We have the Dataset with the prompt and the Reference Summary, it is time to create a Dataset in LangSmith with this information.
### Create the Dataset in Langsmith

The dataset in LangSmith is composed of an input, which is the prompt passed to the model for evaluation, and an output, which should contain what we expect the model to return.

In [14]:
import datetime

In [15]:
import uuid
input_key=['article']
output_key=['highlights']

NAME_DATASET=f"Summarize_dataset_{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

In [17]:
import os

# Print the API key to check its value
print(os.environ.get("LANGCHAIN_API_KEY"))

# If it's not printing or is incorrect, reset it:
os.environ["LANGCHAIN_API_KEY"] = "LANGCHAIN_API_KEY"

# Now retry uploading the dataset.

None


In [21]:
import os

# Retrieve the API key from environment variables
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")

# If the API key is not found or is invalid, prompt the user to enter it
if LANGCHAIN_API_KEY is None or LANGCHAIN_API_KEY == "LANGCHAIN_API_KEY":
    LANGCHAIN_API_KEY = input("Please enter your valid LangChain API key: ")
    os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY  # Store the key in environment variables

# Create the LangSmith client with the updated API key
from langsmith import Client
client = Client(api_key=LANGCHAIN_API_KEY)

Please enter your valid LangChain API key: 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


In [22]:
#This creates the dataset in LangSmith with the content in sample_cnn - If you run this more than once you will get POST errors
dataset = client.upload_dataframe(
    df=sample_cnn,
    input_keys=input_key,
    output_keys=output_key,
    name=NAME_DATASET,
    description="Test Embedding distance between model summarizations",
    data_type="kv"
)

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

In this image, we can see an example from the dataset once it's been registered in LangSmith.

In the Input column, there is the prompt to be sent, while in the Output column, the expected output is stored.

When performing the comparison, the model will be given the prompt, and the Cosine distance between its response and the one stored in the sample dataset will be calculated.
<img src="https://github.com/peremartra/Large-Language-Model-Notebooks-Course/blob/main/img/Martra_Figure_4_2SDL_Dataset.jpg?raw=true">

### Recovering Models From Hugging Face
Let's retrieve both models from HuggingFace. A base T5 model and a model that has been fine-tuned using the training portion of this same dataset to generate summaries.

In [24]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 3.3 MB/s eta 0:00:00


In [25]:
from langchain_community.llms import HuggingFaceHub

In [26]:
from langchain import HuggingFaceHub

In [28]:
import os
from langchain import HuggingFaceHub

# 1. Make sure your Hugging Face API token is set as an environment variable
#    You can set it directly in your notebook or through a .env file
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "HUGGINGFACEHUB_API_TOKEN"  # Replace with your actual token

# 2. Access the token from the environment variable
huggingfacehub_api_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")

# 3. Pass the token to the HuggingFaceHub constructor
summarizer_base = HuggingFaceHub(
    repo_id="t5-base",
    model_kwargs={"temperature": 0, "max_length": 180},
    huggingfacehub_api_token=huggingfacehub_api_token  # Pass the token here
)

In [29]:
summarizer_base = HuggingFaceHub(
    repo_id="t5-base",
    model_kwargs={"temperature":0, "max_length":180},
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN
)

In [30]:
summarizer_finetuned = HuggingFaceHub(
    repo_id="flax-community/t5-base-cnn-dm",
    model_kwargs={"temperature":0, "max_length":180},
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN
)

## Defining Evaluator
The first step is to define an evaluator, where we specify the variables we want to evaluate. In our case, I have chosen to measure only the "embedding_distance."

I've left the "string_distance" as a comment in case you want to conduct a test with two evaluations instead of one.


In [40]:
from langchain.smith import run_on_dataset, RunEvalConfig
# !pip install -q rapidfuzz==3.6.1

In [41]:
!pip install --upgrade langchain

In [49]:
import os
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [51]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.9 MB/s eta 0:00:00


In [52]:
from typing import Optional, List
from pydantic import BaseModel, ConfigDict
from langchain.callbacks.base import BaseCallbackHandler
from openai import OpenAI
import datetime

eval_llm = OpenAI()

class BaseCache(BaseModel):
    model_config = ConfigDict(arbitrary_types_allowed=True)

class Callbacks(BaseModel):
    model_config = ConfigDict(arbitrary_types_allowed=True)
    callbacks: List[BaseCallbackHandler] = []

class RunEvalConfig(BaseModel):
    model_config = ConfigDict(arbitrary_types_allowed=True)
    evaluators: List[str]
    callbacks: Optional[Callbacks] = None
    eval_llm: Optional[OpenAI] = None
    input_key: Optional[str] = "input"
    prediction_key: Optional[str] = "output"
    reference_key: Optional[str] = "highlights"
    custom_evaluators: Optional[List] = None
    batch_evaluators: Optional[List] = None  # Added this line

RunEvalConfig.model_rebuild()

evaluation_config = RunEvalConfig(
    evaluators=["embedding_distance"],
    callbacks=Callbacks(callbacks=[]),
    eval_llm=eval_llm,
    input_key="input",
    prediction_key="output",
    reference_key="highlights",
    batch_evaluators=[]  # Added this line
)

project_name = f"T5-BASE {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

base_t5_results = run_on_dataset(
    client=client,
    project_name=project_name,
    dataset_name=NAME_DATASET,
    llm_or_chain_factory=summarizer_base,
    evaluation=evaluation_config,
)

View the evaluation results for project 'T5-BASE 2025-01-25 10:46:05' at:
https://smith.langchain.com/o/d450b4d5-7ec4-43b1-bc1c-08aac89c9864/datasets/261fef90-f278-4594-a01c-4ad35b271c14/compare?selectedSessions=a9f82f1c-3320-4aaa-aec2-f35156718c2e

View all tests for Dataset Summarize_dataset_2025-01-25 10:28:09 at:
https://smith.langchain.com/o/d450b4d5-7ec4-43b1-bc1c-08aac89c9864/datasets/261fef90-f278-4594-a01c-4ad35b271c14
[------------------------------------------------->] 10/10

In [53]:
#We are using just one of the multiple evaluator avaiable on LangSmith.

evaluation_config = RunEvalConfig(
    evaluators=[
        "embedding_distance",
        #"string_distance"
    ],
)



### Running Evaluator
With the same configuration, we can launch two evaluations on the same dataset. One for each of the chosen models.

In [54]:
project_name = f"T5-BASE {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

base_t5_results = run_on_dataset(
    client=client,
    project_name=project_name,
    dataset_name=NAME_DATASET,
    llm_or_chain_factory=summarizer_base,
    evaluation=evaluation_config,
)

View the evaluation results for project 'T5-BASE 2025-01-25 10:47:36' at:
https://smith.langchain.com/o/d450b4d5-7ec4-43b1-bc1c-08aac89c9864/datasets/261fef90-f278-4594-a01c-4ad35b271c14/compare?selectedSessions=8585f8b2-682d-47db-81e2-e7701223f3e0

View all tests for Dataset Summarize_dataset_2025-01-25 10:28:09 at:
https://smith.langchain.com/o/d450b4d5-7ec4-43b1-bc1c-08aac89c9864/datasets/261fef90-f278-4594-a01c-4ad35b271c14
[------------------------------------------------->] 10/10

In [55]:
#Ignore the error shown below
project_name = f"T5-FineTuned {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

finetuned_t5_results = run_on_dataset(
    client=client,
    project_name=project_name,
    dataset_name=NAME_DATASET,
    llm_or_chain_factory=summarizer_finetuned,
    evaluation=evaluation_config,
)

View the evaluation results for project 'T5-FineTuned 2025-01-25 10:47:43' at:
https://smith.langchain.com/o/d450b4d5-7ec4-43b1-bc1c-08aac89c9864/datasets/261fef90-f278-4594-a01c-4ad35b271c14/compare?selectedSessions=ee5d1aa6-513d-4c94-b670-d5ac3b86969d

View all tests for Dataset Summarize_dataset_2025-01-25 10:28:09 at:
https://smith.langchain.com/o/d450b4d5-7ec4-43b1-bc1c-08aac89c9864/datasets/261fef90-f278-4594-a01c-4ad35b271c14
[------------------------------------------------->] 10/10

<img src="https://github.com/peremartra/Large-Language-Model-Notebooks-Course/blob/main/img/Martra_Figure_4_2SDL_Tests.jpg?raw=true">

In the image below you can see the comparision between two tests.
<img src="https://github.com/peremartra/Large-Language-Model-Notebooks-Course/blob/main/img/Martra_Figure_4_2SDL_CompareTestst.jpg?raw=true">

Well, since it has been so straightforward, why don't we try to make the comparison with an OpenAI model?

In [57]:
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 2.3 MB/s eta 0:00:00


In [58]:
from langchain_openai import OpenAI
open_aillm=OpenAI(temperature=0.0)

In [59]:
project_name = f"OpenAI {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

finetuned_t5_results = run_on_dataset(
    client=client,
    project_name=project_name,
    dataset_name=NAME_DATASET,
    llm_or_chain_factory=open_aillm,
    evaluation=evaluation_config,
)

View the evaluation results for project 'OpenAI 2025-01-25 10:49:20' at:
https://smith.langchain.com/o/d450b4d5-7ec4-43b1-bc1c-08aac89c9864/datasets/261fef90-f278-4594-a01c-4ad35b271c14/compare?selectedSessions=990a2636-252b-440d-9433-edd94b7790b4

View all tests for Dataset Summarize_dataset_2025-01-25 10:28:09 at:
https://smith.langchain.com/o/d450b4d5-7ec4-43b1-bc1c-08aac89c9864/datasets/261fef90-f278-4594-a01c-4ad35b271c14
[------------------------------------------------->] 10/10

<img src="https://github.com/peremartra/Large-Language-Model-Notebooks-Course/blob/main/img/Martra_Figure_4_2SDL_CompareOpenAI_HF.jpg?raw=true">

The experiment with the OpenAI model has yielded the best results. But, be aware! As we can see, there is a cost involved since we are using an API, and it needs to be paid for.

Another crucial piece of information is that we can view performance data for the models. This data could also be useful for minimally evaluating our inference server.